## Test a clean install

- conda installs 
- pip jupyter install
- document the .env file

## Restructure

- read.py (interfaces to phonon data)
    - reads in phonon dos data and transforms into an ASE atoms object with dos data stored with the "info" attribute
    - reads in from: materials project API, phonopy output file, ASE phonon object.
    - returns dos_data as a dataframe, and information about the system (e.g. composition)
- analysis.py (manipulate the raw data, physics inspired or to reduce it)
    - system class initialised with ASE atoms object and a pandas 2D dataframe with frequencies and dos.
    - any post-processing / fitting
- sonification.py (strauss)
    - various strauss sonifications initialised with system object
    - parent class can be output to sounddevice or wav file via methods
- main.py (strauss)
    - gets data from read.py
    - uses this to create an system object
    - uses system to create a sonification object
    - use sonification method to output to sounddevice or wav
- app.py
    - Flask app functionality
    - runs main.py

## Implement

- the strauss sonification in the web app

In [1]:
import sounddevice
import numpy as np
import math
import time
import yaml
from scipy import constants

import os
import argparse

phonon_mesh_filepath = './data/BaS_Fm3m/mesh.yaml'
sample_rate = 44100
min_audible = 20# minimum audible frequency in herz
max_audible = 8E3 # maximum audible frequency in herz

In [2]:
def frequencies_from_mp_id(mp_id):
    """return phonon frequencies at gamma point from for a material hosted on the Materials Project ().
    Material is identified using unique ID number. Note that to use this feature you need a Materials
    Project API key (https://materialsproject.org/api)."""
    import mp_api
    from mp_api.client import MPRester
    from dotenv import load_dotenv
    load_dotenv () # use python-dotenv library for storing secrets in a .env file in project route (or at another path that is specified here)

    with MPRester(os.getenv('MP_API_KEY')) as mpr:
        try:
            bs = mpr.get_phonon_bandstructure_by_material_id(mp_id)
        except:
            print("this materials project entry does not appear to have phonon data")
            pass
    return bs
    print("extracting frequencies for qpoint {}".format(bs.qpoints[0].cart_coords))

    phonon_frequencies = list(bs.bands[:,0])
    phonon_frequencies = process_imaginary(phonon_frequencies)
    print("phonon frequencies are (THz):", phonon_frequencies)

    return phonon_frequencies

In [3]:
phonon_docs = frequencies_from_mp_id("mp-7548")

In [13]:
phonon_docs.to_pmg.bands[:,0]

array([0.        , 0.        , 0.        , 1.74501577, 1.76241428,
       1.79155713, 2.27649034, 2.37765811, 2.56556928, 2.66184605,
       2.80429073, 2.90040781, 3.03372632, 3.13715636, 3.34913151,
       3.36353303, 3.51090568, 3.89215787, 3.95236191, 4.34024086,
       6.6999855 , 6.72189244, 7.28162931, 7.31176969])

In [35]:
mpr.materials.phonon.search?

Signature:
mpr.materials.phonon.search(
    material_ids: 'str | list[str] | None' = None,
    num_chunks: 'int | None' = None,
    chunk_size: 'int' = 1000,
    all_fields: 'bool' = True,
    fields: 'list[str] | None' = None,
) -> 'list[PhononBSDOSDoc] | list[dict]'
Docstring:
Query phonon docs using a variety of search criteria.

Arguments:
    material_ids (str, List[str]): A single Material ID string or list of strings
        (e.g., mp-149, [mp-149, mp-13]).
    num_chunks (int): Maximum number of chunks of data to yield. None will yield all possible.
    chunk_size (int): Number of data entries per chunk.
    all_fields (bool): Whether to return all fields in the document. Defaults to True.
    fields (List[str]): List of fields in PhononBSDOSDoc to return data for.
        Default is material_id, last_updated, and formula_pretty if all_fields is False.

Returns:
    ([PhononBSDOSDoc], [dict]) List of phonon documents or dictionaries.
File:      ~/miniconda3/envs/singing-materia

In [27]:
from dotenv import load_dotenv
import os
load_dotenv ()
import mp_api
from mp_api.client import MPRester
mpr = MPRester(os.getenv('MP_API_KEY'))

In [28]:
mpr.materials.phonon.search?

Signature:
mpr.materials.phonon.search(
    material_ids: 'str | list[str] | None' = None,
    num_chunks: 'int | None' = None,
    chunk_size: 'int' = 1000,
    all_fields: 'bool' = True,
    fields: 'list[str] | None' = None,
) -> 'list[PhononBSDOSDoc] | list[dict]'
Docstring:
Query phonon docs using a variety of search criteria.

Arguments:
    material_ids (str, List[str]): A single Material ID string or list of strings
        (e.g., mp-149, [mp-149, mp-13]).
    num_chunks (int): Maximum number of chunks of data to yield. None will yield all possible.
    chunk_size (int): Number of data entries per chunk.
    all_fields (bool): Whether to return all fields in the document. Defaults to True.
    fields (List[str]): List of fields in PhononBSDOSDoc to return data for.
        Default is material_id, last_updated, and formula_pretty if all_fields is False.

Returns:
    ([PhononBSDOSDoc], [dict]) List of phonon documents or dictionaries.
File:      ~/miniconda3/envs/singing-materia